In [91]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re


%matplotlib inline

### Challenge
Determine the five most common journals and the total articles for each. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal . 

In [92]:
# Here is the "dirty" data file that I need to clean
dirty = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv',encoding='"ISO-8859-1"')
dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
PMID/PMCID                                             1928 non-null object
Publisher                                              2127 non-null object
Journal title                                          2126 non-null object
Article title                                          2127 non-null object
COST (£) charged to Wellcome (inc VAT when charged)    2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [93]:
df=dirty.copy()
df= df.rename(columns={'PMID/PMCID': "ID", 'Publisher':'Publisher','Journal title': 'Journal', 'Article title': 'Article', 
                       'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost'})
df.head()

,ID,Publisher,Journal,Article,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 5 columns):
ID           1928 non-null object
Publisher    2127 non-null object
Journal      2126 non-null object
Article      2127 non-null object
Cost         2127 non-null object
dtypes: object(5)
memory usage: 83.2+ KB


In [95]:
#REMOVE NULL VALLUE IN JOURNAL
df=df[df['Journal'].notnull()][['ID','Publisher','Journal','Article','Cost']]

In [96]:
#Now I want to see how dirty the journal names are before I proceed
df['Journal'].value_counts()

PLoS One                                                                92
PLoS ONE                                                                62
Journal of Biological Chemistry                                         48
Nucleic Acids Research                                                  21
Proceedings of the National Academy of Sciences                         19
Human Molecular Genetics                                                18
PLoS Neglected Tropical Diseases                                        18
Nature Communications                                                   17
PLoS Pathogens                                                          15
Neuroimage                                                              15
PLoS Genetics                                                           15
NeuroImage                                                              14
BMC Public Health                                                       14
PLOS ONE                 

There are clearly a lot of mistakes in the entries of the Journal titles (i.e. duplications due to misspellings, acronyms, and also extra whitespace, extraneous punctuation and capitalization, etc.). There are also some journals like Plos One that have other versions or editions from that Publisher. For the purpose of this challege, I will be treating these separate. 



In [97]:
df['ID'].head(20)

0                              NaN
1                       PMC3679557
2            23043264  PMC3506128 
3              23438330 PMC3646402
4             23438216 PMC3601604 
5                       PMC3579457
6                       PMC3709265
7             23057412 PMC3495574 
8                PMCID: PMC3780468
9                PMCID: PMC3621575
10               PMCID: PMC3739413
11               PMCID: PMC3530961
12               PMCID: PMC3624797
13                      PMC3413243
14                      PMC3694353
15                      PMC3572711
16                        22610094
17               PMCID: PMC3586974
18    23455506  PMCID: PMC3607399 
19      PMID: 24015914 PMC3833349 
Name: ID, dtype: object

### Dealing with Journal Dirt
1. Strip Whitespace
2. Convert case 
3. Remove prepositions and other misc. characters
4. Fix Mistakes in Spelling
5. Replace remaining dirty strings with clean strings (i.e. abbreviations and other inconsistencies)
6. Remove currency symbols from cost and convert as float. 

In [98]:
df['clean_journal'] =df['Journal'].str.strip()
df['clean_journal'] =df['Journal'].str.upper()
df['clean_journal'].value_counts()

PLOS ONE                                                    190
JOURNAL OF BIOLOGICAL CHEMISTRY                              53
NEUROIMAGE                                                   29
PLOS PATHOGENS                                               24
PLOS GENETICS                                                24
NUCLEIC ACIDS RESEARCH                                       23
PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCES              20
PLOS NEGLECTED TROPICAL DISEASES                             20
HUMAN MOLECULAR GENETICS                                     19
NATURE COMMUNICATIONS                                        19
BMC PUBLIC HEALTH                                            14
BRAIN                                                        14
MOVEMENT DISORDERS                                           13
JOURNAL OF NEUROSCIENCE                                      12
BIOCHEMICAL JOURNAL                                          12
DEVELOPMENTAL CELL                      

From this, we can already see that some of the Journal like Plos One have been cleaned and grouped.
Now we have 900 unique journal titles, versus almost 1000 prior. We are not done yet, at there is still cleaning. 
There are still misspellings and issues with puncuation. For example, there is a Plos One as well as a Plosone

Next I should remove the punctuation and other symbols, so I can more easily look for duplicates or errors. 

In [99]:
df['clean_journal'] = df['clean_journal'].str.replace('&', 'AND')
df['clean_journal'] = df['clean_journal'].str.replace('THE ', '')
df['clean_journal'] = df['clean_journal'].str.replace('.', '')
df['clean_journal'] = df['clean_journal'].str.replace('OF ', '')
df['clean_journal'] = df['clean_journal'].str.replace('FOR ', '')
df['clean_journal'] = df['clean_journal'].str.replace(' IN ', ' ')
df['clean_journal'] = df['clean_journal'].str.replace(':', '')
df['clean_journal'] = df['clean_journal'].str.replace(',', '')
df['clean_journal'] = df['clean_journal'].str.replace(' - ',' ')
df['clean_journal'] =df['clean_journal'].str.strip()

In [100]:
df['clean_journal'].value_counts()

PLOS ONE                                     190
JOURNAL BIOLOGICAL CHEMISTRY                  61
NEUROIMAGE                                    29
NUCLEIC ACIDS RESEARCH                        26
PLOS PATHOGENS                                24
PLOS GENETICS                                 24
PROCEEDINGS NATIONAL ACADEMY SCIENCES         22
PLOS NEGLECTED TROPICAL DISEASES              20
HUMAN MOLECULAR GENETICS                      19
NATURE COMMUNICATIONS                         19
JOURNAL NEUROSCIENCE                          15
MOVEMENT DISORDERS                            15
BRAIN                                         14
BMC PUBLIC HEALTH                             14
BIOCHEMICAL JOURNAL                           14
DEVELOPMENTAL CELL                            12
JOURNAL GENERAL VIROLOGY                      11
FASEB JOURNAL                                 11
CURRENT BIOLOGY                               11
MALARIA JOURNAL                               10
BMJ                 

After inspecting the data, I want to go in an replace items that are still incorrect, which are mostly spelling errors

In [101]:
#other msc. misppellings
df['clean_journal'] = df['clean_journal'].str.replace('AGFENTS','AGENTS')
df['clean_journal'] = df['clean_journal'].str.replace('ANTIMICOBIAL','ANTIMICROBIAL')
df['clean_journal'] = df['clean_journal'].str.replace('BEHAVIOUR ','BEHAVIOR ')
df['clean_journal'] = df['clean_journal'].str.replace('BEHAVIOURAL ','BEHAVIORAL ')
df['clean_journal'] = df['clean_journal'].str.replace('JNL ','JOURNAL ')
df['clean_journal'] = df['clean_journal'].str.replace('BRT ','BRITISH ')
df['clean_journal'] = df['clean_journal'].str.replace('INT ','INTERNATIONAL ')
df['clean_journal'] = df['clean_journal'].str.replace('JOUNRAL ','JOURNAL ')
df['clean_journal'] = df['clean_journal'].str.replace('JOUNAL ','JOURNAL ')
df['clean_journal'] = df['clean_journal'].str.replace('JOURAL ','JOURNAL ')
df['clean_journal'] = df['clean_journal'].str.replace('JOURNALS','JOURNAL')
df['clean_journal'] = df['clean_journal'].str.replace('BIOOHYSICA','BIOPHYSICA')
df['clean_journal'] = df['clean_journal'].str.replace('BIINFORMATICS','BIOINFORMATICS')
df['clean_journal'] = df['clean_journal'].str.replace('BIOLOGICIAL','BIOLOGICAL')
df['clean_journal'] = df['clean_journal'].str.replace('BIOL CHEM','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('BIOL CHEMISTRY','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('J BIOL CHEM','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('J BIOL CHEMISTRY','BIOLOGICAL CHEMISTRY')
df['clean_journal'] = df['clean_journal'].str.replace('BMS ','BMC ')
df['clean_journal'] = df['clean_journal'].str.replace('OPTHALMOLOGY','OPHTHALMOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('ONLINE','')
df['clean_journal'] = df['clean_journal'].str.replace('PRINT','')
df['clean_journal'] = df['clean_journal'].str.replace('CURR BIOL','CURRENT BIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('EPIGENTICS','EPIGENETICS')
# df['clean_journal'] = df['clean_journal'].str.replace('EPIDEMIOL','EPIDEMIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace(' J ',' JOURNAL ')
df['clean_journal'] = df['clean_journal'].str.replace('HEPTOLOGY','HEPATOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('MOL ','MOLECULAR ')
df['clean_journal'] = df['clean_journal'].str.replace('INYERNATIONAL','INTERNATIONAL')
df['clean_journal'] = df['clean_journal'].str.replace('PLOSONE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PLOS  ONE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PUBLIC LIBRARY SCIENCE ONE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PUBLIC LIBRARY SCIENCE','PLOS ONE')
df['clean_journal'] = df['clean_journal'].str.replace('PLOS 1','PLOS ONE')
# df['clean_journal'] = df['clean_journal'].str.replace('BIOCHEM','BIOCHEMICAL')
df['clean_journal'] = df['clean_journal'].str.replace('MOL BIOL','MOLECULAR BIOLOGY')
df['clean_journal'] = df['clean_journal'].str.replace('J MOLECULAR','JOURNAL MOLECULAR')
df['clean_journal'] = df['clean_journal'].str.replace(' OD ',' ')
df['clean_journal'] = df['clean_journal'].str.replace('EXPERIEMENTS','EXPERIMENTS')
df['clean_journal'] = df['clean_journal'].str.replace('EXPERMIMENTS','EXPERIMENTS')
df['clean_journal'] = df['clean_journal'].str.replace('NEUROLMAGE','NEUROIMAGE')
df['clean_journal'] = df['clean_journal'].str.replace('ACID ','ACIDS ')
df['clean_journal'] = df['clean_journal'].str.replace('AMERICAL','AMERICAN')
df['clean_journal'] = df['clean_journal'].str.replace('BRITSH','BRITISH')



df['clean_journal'] =df['clean_journal'].str.strip()

For all of the other errors with the entries (whether from abbreviations, or other inconsistencies) I will manually change the title with lambda expressions. Note, I am not changing all errors, particularly the dirt for Journals that do not have a high count in the dataset to begin with. 

In [102]:
#ACTA journals from International Union of Crystallography 
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA D' if 'ACTA CRYSTALLOGRAPHICA SECTION D' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA D' if 'ACTA CRYSTALLOGRAPHY D' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA F' if 'ACTA CRYSTALLOGRAPHICA SECTION F' in x else x)
#ACTA Neuropathology
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ACTA NEUROPATHOLOGICA' if 'ACTA NEUROPATHOL' in x else x)
#AIDS Journal by Wolters Kluwer
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'AIDS JOURNAL' if 'AIDS' == x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'AIDS JOURNAL' if 'AIDS UK' == x else x)
#AMERICAN JOURNAL MEDICAL GENETICS
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'AMERICAN JOURNAL MEDICAL GENETICS' if \
                                                'AMERICAN JOURNAL MEDICAL GENETICS PART A' in x else x)
#ANGEWANDTE CHEMIE
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION' if \
                                                'ANGEWANDE CHEMIE' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION' if \
                                                'ANGEWANDTE CHEMIE' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'ANGEWANDTE CHEMIE INTERNATIONAL EDITION' if \
                                                'ANGEW CHEMS INTERNATIONAL ED' in x else x)
#DEVELOPING WORLD BIOETHICS
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'DEVELOPING WORLD BIOETHICS' if 'DEV WORLD BIOETH' in x else x)
#AMERICAN JOURNAL TROPICAL MEDICINE AND HYGIENE
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'AMERICAN JOURNAL TROPICAL MEDICINE AND HYGIENE' \
                                                if 'AM JOURNAL TROP MED HYG' in x else x)
#TROPICAL MEDICINE AND INTERNATIONAL HEALTH
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'TROPICAL MEDICINE AND INTERNATIONAL HEALTH' \
                                                if 'TROP MED INTERNATIONAL HEALTH' in x else x)
#BIOCHEMICAL SOCIETY TRANSACTIONS
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'BIOCHEMICAL SOCIETY TRANSACTIONS' \
                                                if 'BIOCHEM SOC TRANS' in x else x)
#BIOINFORMATICS 
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'BIOINFORMATICS' if 'BIOINFORMATICS ONLINE' in x else x)
#BIOLOGY OPEN
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'BIOLOGY OPEN' if 'BIOL OPEN' in x else x)
#JOURNAL OF BIOLOGICAL CHEMISTRY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL OF BIOLOGICAL CHEMISTRY' \
                                                if 'BIOLOGICAL CHEMISTRY' in x else x)
#CELL JOURNAL
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'CELL JOURNAL' if 'CELL' == x else x)
#CELL DEATH AND DIFFERENTIATION
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'CELL DEATH AND DIFFERENTIATION' \
                                                if 'CELL DEATH DIFFERENTIATION' in x else x)
#CHILD CARE HEATH AND DEVELOPMENT
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'CHILD CARE HEATH AND DEVELOPMENT' \
                                                if 'CHILD CARE HEALTH DEVELOPMENT' in x else x)
#BIOCHIMICA ET BIOOHYSICA ACTA MOLECULAR BASIS DISEASE
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'BIOCHIMICA ET BIOPHYSICA ACTA MOLECULAR BASIS DISEASE' \
                                                if 'BBA ' in x else x)
#CLINICAL INFECTIOUS DISEASES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'CLINICAL INFECTIOUS DISEASES' \
                                                if 'CLINICLA INFECTIOUS DISEASE' in x else x)
#JOURNAL INFECTIOUS DISEASES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL INFECTIOUS DISEASES' \
                                                if 'J INFECT DIS' in x else x)
#EMBO JOURNAL
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'EMBO JOURNAL' if 'EMBO' == x else x)
#EUROPEAN JOURNAL IMMUNOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'EUROPEAN JOURNAL IMMUNOLOGY' if 'EUR JOURNAL IMMUNOL' in x else x)
#FEBS JOURNAL
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'FEBS JOURNAL' if 'FEBS J' == x else x)
#JOURNAL CLINICAL VIROLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL CLINICAL VIROLOGY' if 'VIROLOGY' == x else x)
#HUMAN MUTATION
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'HUMAN MUTATION' if 'HUMAN MUTATION' in x else x)
#JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL ACQUIRED IMMUNE DEFICIENCY SYNDROMES'\
                                                if 'ACQUIRED IMMUNE DEFICIENCY' in x else x)
#JOURNAL IMMUNOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL IMMUNOLOGY' if 'J IMMUNOL' in x else x)
#JOURNAL CELL SCIENCE
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL CELL SCIENCE' if 'J CELL SCI' in x else x)
#JOURNAL CLINICAL MICROBIOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL CLINICAL MICROBIOLOGY' if 'J CLIN MICROBIOL' in x else x)
#JOURNAL MEDICINAL CHEMISTRY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL MEDICINAL CHEMISTRY' if 'J MED CHEM' == x else x)
#JOURNAL MECHANISMS AGEING AND DEVELOPMENT
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL MECHANISMS AGEING AND DEVELOPMENT' \
                                                if 'MECHANISMS AGEING AND DEVELOPMENT' in x else x)
#JOURNAL MICROBIOLOGY
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'JOURNAL MICROBIOLOGY' if 'MICROBIOLOGY' == x else x)
#PARASITES AND VECTORS
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PARASITES AND VECTORS' if 'PARASIT VECTORS' == x else x)
#PROCEEDINGS NATIONAL ACADEMY SCIENCES
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' if 'PNAS' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' if 'PROCEDDINGS' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' \
                                                if 'PROC NATL ACAD SCI U S A' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' \
                                                if 'PROCEEDINGS NATIONAL ACADEMY SCIENCES UNITED STATES AMERICA' in x else x)
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PROCEEDINGS NATIONAL ACADEMY SCIENCES' \
                                                if 'PROCEEDINGS NATIONAL ACADEMY SCIENCES USA' in x else x)
#Public Library of Science
df['clean_journal'] = df['clean_journal'].apply(lambda x: 'PLOS ONE' if 'PLOS' == x else x)

In [103]:
df['clean_journal'] =df['clean_journal'].str.strip()
df['clean_journal'].value_counts()

PLOS ONE                                                                        213
JOURNAL OF BIOLOGICAL CHEMISTRY                                                  73
PROCEEDINGS NATIONAL ACADEMY SCIENCES                                            37
NEUROIMAGE                                                                       31
NUCLEIC ACIDS RESEARCH                                                           29
PLOS GENETICS                                                                    24
PLOS PATHOGENS                                                                   24
HUMAN MOLECULAR GENETICS                                                         21
PLOS NEGLECTED TROPICAL DISEASES                                                 20
NATURE COMMUNICATIONS                                                            19
BRAIN                                                                            18
MOVEMENT DISORDERS                                                          

OK, now my Journal are (mostly) clean--clean enough to find the most popular journals. 


In [104]:
df['clean_journal'].value_counts().head(5)

PLOS ONE                                 213
JOURNAL OF BIOLOGICAL CHEMISTRY           73
PROCEEDINGS NATIONAL ACADEMY SCIENCES     37
NEUROIMAGE                                31
NUCLEIC ACIDS RESEARCH                    29
Name: clean_journal, dtype: int64

These are the 5 most common journals by count. Now I need to remove the symbols from costs and convert to float. 

In [105]:
df['Cost'] = df['Cost'].str.lstrip('£').str.rstrip('$').astype(float)

In [106]:
df.Cost.describe()

count      2126.000000
mean      24072.451609
std      146895.027687
min           0.000000
25%        1280.000000
50%        1883.860000
75%        2321.230000
max      999999.000000
Name: Cost, dtype: float64

There are clearly high and low Cost per Articles in the data set. I am going to remove any costs 1.5 * IQR above the 3rd quartile(no observations are 1.5 * IQR below 1st quartile)

#Can i winsorize? Is it ok to remove the outliers above 1.5 time IQR

In [107]:
df[df['Cost']>3883] = None

In [ ]:
df_top5 = df.groupby('clean_journal')['revenue_M'].agg(['mean','count']).\
                                  rename(columns={'mean':'avg_revenue','count':'actor_1_count'})

In [108]:
df.Cost.describe()

count    2036.000000
mean     1774.272191
std       693.854948
min         0.000000
25%      1260.000000
50%      1834.770000
75%      2282.632500
max      3882.410000
Name: Cost, dtype: float64

### Finding Stats


In [109]:
df_stats = df['clean_journal'].value_counts().to_frame()
df_stats.rename(columns={'clean_journal': 'Count'}, inplace=True)
df_stats['Mean'] = None
df_stats['Median'] = None
df_stats['Std'] = None
df_stats.head(5)

,Count,Mean,Median,Std
PLOS ONE,204,None,None,None
JOURNAL OF BIOLOGICAL CHEMISTRY,71,None,None,None
PROCEEDINGS NATIONAL ACADEMY SCIENCES,36,None,None,None
NEUROIMAGE,31,None,None,None
NUCLEIC ACIDS RESEARCH,29,None,None,None


In [110]:
for title in df_stats.index:
    df_stats.loc[title, 'Mean'] = df[df['clean_journal']==title]['Cost'].mean()
    df_stats.loc[title, 'Median'] = df[df['clean_journal']==title]['Cost'].median()
    df_stats.loc[title, 'Std'] = df[df['clean_journal']==title]['Cost'].std()  

df_stats.head(5)

,Count,Mean,Median,Std
PLOS ONE,204,937.966,897.61,192.877
JOURNAL OF BIOLOGICAL CHEMISTRY,71,1378.14,1311.73,388.554
PROCEEDINGS NATIONAL ACADEMY SCIENCES,36,772.919,732.01,370.192
NEUROIMAGE,31,2212.18,2326.43,273.193
NUCLEIC ACIDS RESEARCH,29,1162.34,852,442.151


## Future Work/Changes I would make
I would obviously want to do further cleaning of the journal titles. 
Originally, I was going to remove 'Journal' completely to filter for duplicates, but I noticed this would give me the same title for multiple publications in some instances. I would want to first clean the Publishers, and then iterate through the duplicate journal titles and count them as separate if they have different Publishers listed. 

I would also want to convert the costs listed in $ to pounds, so all costs are in same currency. 

Also, there seems to be issues with the ID formatting. Some IDs are listed as PMID and others as PMCID, while there are even others that have both. There are even instances of duplicate IDs (due to erros in other fields, such as Article Title).
I would want to clean these to better understand how this column contributes to overall dirtiness of the data set. 